<a href="https://colab.research.google.com/github/BenoCarrizo/Proyecto_Titulo/blob/master/BERT_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 1: Importar las dependencias

Ahora se usara BERT para crear una capa de embedding.
El dataset y el modelo seran los mismo. La diferencia estara en que la red neuronal convolucional, en vez de utilizar la capa de embedding creada por nosotros usando tensorflow, se usara la capa de embedding de BERT, de modo que no vamos a entrenar los pesos de las variables que se corresponden a la capa de embedding, sino que sera BERT quien las maneje por nosotros. En pocas palabras, se obtendra una representacion vectorial de nuestras palabras optimizada por BERT y se utilizara para ver si nuestro modelo mejora con respecto a como lo teniamos en la seccion precedente.
Esto de puede usar en cualquier proyecto de NLP, usar el la capa de embedding de BERT.

Lo que va a cambiar va a ser una parte de la limpieza (preprosesado), ya que habra que codificar la frase del modo que espera la capa de embedding y por supuesto, el entrenamiento y la evaluacion va a ser el mismo.

Hay dos cosas grandes que hay que hacer para adaptar el modelo. La primera es la entrada, porque la primera de las capas de nuestro modelo ahora va a ser una capa BERT.

En la creacion del modelo, hay que cambiar la capa de embedding por defecto, para usar la capa de embedding de BERT.

* El primer cambio lo hacemos en la fase de Tokenizacion, ya que necesitamos añadir algunos tokens a las entradas de BERT. El primero es CLS el cual va al inicio, se usa para probelams de clasificacion, y un token SEP de separacion entre frases.
Para la funcion encode_sentence, esta ahora devolvera el token CLS al inicio, luego la tokenizacion de la frase y por ultimo  el separador SEP.

* El segundo cambio lo hacemos en la seccion "Creacion de dataset". La capa de BERT necesita tres entredas, formadas por tres tipos de tokens diferentes, y estos tokens se generan a partir de las frases (elementos tokenizados). Entonces, una vez que tenemos la frase tokenizada, necesitamos crear esos tres tokens, y para ello necesitaremos dos cosas. La primera es una lista de mascaras. Esto le va a indicar a BERT cuales son los tokens que no tiene que utilizar en la transformacion de datos para convertirlos en vectores de embedding, porque son valores desconocidos que tiene que predecir.  Si uno de los elementos de la frase esta marcado por la mascara, entonces ese sera el valor que queremos predecir (no importa cuales son sus coordenadas, ya que son desconocidas, son las que se quieren predecir)
No hemos usado padding manual todavia.
Tambien hara falta saber quien pertenece a la primera frase y quien pertenece a la segunda frase. El tercer tipo de enetrada era el segment input, que basicamente era una lista de ceros y unos, indicando si era la primera frase, en cuyo caso los tokens seran los ceros.

Entonces la primera funcion, get_ids, la cual, a partir de una lista de tokens de palabras tokenizadas, palabras separadas, mas el CLS y mas el separador SEP, el tokenizer se encargara de convertir los tokens a ids. Con esto devolvemos la lista de ids para cada uno de los tokens, la version del formto numero en lugar de la version en formato string que habiamos dejado lista a partir del resultado de la linea anterior.

La segunda funcion (get_mask) es la de aplicar la mascara correspondiente a los tokens de padding. En esta funcion lo que se hara sera buscar, dentro de la lista de tokens, la existencia del token PAD. Cuando encontremos este token, significa que ese elemento no nos interesa. La funcion busca los tokens que no son iguales a PAD, devolviendo valores 1 para los que no son PAD y 0s para los que si son PAD. Basicamente esta funcion devolvera una lista de la misma longitud que la de tokens que le haya pasado como entrada, pero con ceros y unos.

Y la tercera funcion indicara si un token pertenece a la primera frase o a la segunda frase. Esto lo haresmos gracias a ubicar el token de separacion en cuestion, ya que hasta encontrar el token SEP, se utilizaran ceros para indicar que el fragmento de frase corresponde al primer trozo, y despues de encontrar el token de separacion, colocaremos unos en este caso.

Ahora hay que cambiar es sorted_all, la cual ahora generara vectores de 4 alementos para cada fila, y cada uno es una tupla. En primer lugar se ponen los identificadores para la frase (sentence label(sent_lab) es el iterador de data_with_len, por tanto, en sentence label nos encontramos con elementos que tienen tres columnas, en la cero esta la frase, en la uno esta la etiqueta y en la dos esta la longitud). Luego se le pone la mascara (get_mask) para la misma frase y los segmentos (get_segment) para la propia frase. De este modo empaquetamos en la tupla, en primera posicion ([0]) esos tres elementos (identificadores, mascaras y segmentos) en una lista, y en la segunda ([1])posicion de la tupla la etiqueta, por lo que sent_lab[1] nos dara la etiqueta para esa frase que acabo de triplicar su tamaño, gracias a los identificadores, tokens, la mascara y los segmentos de frase 01. Cabe mensionar que los ids, las mascaras y los segmentos entraran en el embedding de BERT junto con la etiqueta.

Llegado a este punto, ya se hicieron todos los cambios necesarios en la entrada para poder utilizar BERT como capa de embedding en nuestra red neuronal convolucional, por lo que la segunda fase esta lista.




### La capa de embedding de BERT
Lo que sigue es modificar el modelo (construccion del modelo) y evaluar su performance para ver que tanto mejor se comporta con respecto a la capa de medir previa que habiamos creado utilizando tensorflow. 

Cuando aplicaquemos la bert.layer, veremos que salen dos tersones. El primero tiene dimension (1,768), esto significa que es un lote de una sola frase, y esa frase contiene 768 elementos, esa es la dimension del embedding, y esto se corresponde al token CLS. Esta es la informacion que se utilizaria en tareas de clasificacion (representacion vectorial de una frase).
El segundo tensor es de dimension (1, 6, 768), el cual es de un solo lote, pero ahora tiene coordenadas de 768 valores para cada token, osea, para cada palabra.
Este tensor no corresponde a la codificacion de toda la frase, sino, con la codificacion de todos y cada uno de los elementos de las palabras tokenizadas de nuestra frase. Y 1,6,768 , 1 corresponde al lote, 6 corresponde a las palabras del ejemplo (CLS the roses are red SEP), y 768 corresponde a las coordenadas que mejor se ajustan a cada una de ellas (un vector para cada token). La segunda se usa cuando se necesita llegar a la especificacion del token o palabras, por separado, o sea, una aplicacion de palabra a palabra (de token a token, o sea, la representacion vectorial de cada una de las palabras).
#### EJEMPLO

my_sent = ["[CLS]"] + tokenizer.tokenize("Roses are red.") + ["[SEP]"]
print(my_sent)

bert_layer([tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32), 0)])

## Cambio de la capa de embedding en el modelo

Primero cambiamos el nombre de la clase DCNNBERTEmbedding, y como parametro, ya no esta el tamaño del vocabulario ni la dimension del embedding, ya que la capa de bert ya sabe cual es la dimension del embedding, y tambien ya sabe cual es el tamaño del vocabulario porque utiliza su propio tokenizador.

Otro cambio se produjo en la capa de embeddings (self.bart_layer = hub.KerasLeyer), ya que ya no es la de tensorflow, en vez de eso, ahora se usa algo parecido a como se uso la primera capa de bert para el tokenizador. Tambien se usa el trainable = false, ya que se quiere utilizar el embedding ya entrenado por google (si fuera true se haria un finetunning).

Otro cambio se produce en la funcion call, ya que ahora llama a otra funcion nueva llamada embed_with_bert, la cual recibe el parametro all_tokens (token de entrada. Recordar que vamos a utilizar bloques (batches) de tokens), y lo que se hace es, usando la capa ver definida en init (__init__) para darle la forma correcta, ya que, para cada grupo de tokens, se deben obtener los identificacdores, la mascara y el token de frase en el orden adecuado. Entonces basicamente lo que se hace es entrar a all_tokens, se construye en una lista de python accediendo, de todos los tokens, a todo el bloque (a todas las frases que vengan en el bloque). El elemento 0 (all_tokens[:,0,:]) son los token que correspode a las palabras de cada frase. El elemento 1 (all_tokens[:,1,:]), que es el de tokens de mascara, quiere decir que de todo el bloque obtenemos unicamenten la dimension que corresponde a la cascara y para todas y cada una de las palabras. Y lo mismo sucede con el token de frase, o sea, con el 2 (all_tokens[:,2,:]). 

 def embed_with_bert(self, all_tokens):
        _, embs = self.bert_layer([all_tokens[:, 0, :],
                                   all_tokens[:, 1, :],
                                   all_tokens[:, 2, :]])
        return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)

        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output


Una vez que tenemos la forma, construimos la entrada, tal como lo hemos visto hates (ejemplo capa de embedding).
Recordar que nos da una salida doble (_,embs). La primera respuesta (_) se usa cuando se quiere una analisis de la frase en  global (vector de 768 que representa la frase en todo si contexto), y la segudas parte es la que se corresponde con los tokens, las palabras individuales de mapeados a en dimension 768. En el primero ponemos _ porque no nos aporta nada, ya que lo que se quiere es analizar palabra por palabra y por tanto se retornan los embeddings que es el segundo de los tensores que devuelven la capa de bert de la capa de embedding. 

### Entrenamiento 

En esta fase no cambia casi nada, solo en la parte donde se llama a la red neuronal, ya que no necesitamos la dimension del vocabulario ni la dimension del embedding, por lo que se debe enviar menos informacion.
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2


                        


```

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)
```

## Resumen

* La entrada de BERT requiere de una entrada triple de (get_ids, get_mask, get_segments).

* Cuando se llama a la capa de embedding se obtienen dos respuestas diferentes (_, embs), un vector que representa la frase ene su totalidad y otro vector que 
es una lista de vectores, cada uno de los cuales representa cada una de las palabras de los tokens utilizados (para nuestro caso usamos la segunda).

* Si se quiere usar BERT como herramienta se debe recordar poner la false en trainable.



```
self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=False)
```
* Y una observacion es que al final, en la fase de entrenamiento, prescindimos de hiperaparametros, por lo que solo se usan unos pocos.


In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
import time

from google.colab import drive

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 177 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=73ef72a59297643242a77c26a710ba7e1b93dd47f0f2e80553e22850146f49d1
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=45eba42ce2d3d154f1d4f31e8add784107f45c191fd34ec6e03cd7154b2d9117
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=de67dad60dc2a1b9de6b8dea3731dfae03f91b869afbfdb5dc0fd2d07aeaaf5b
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow p

In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Fase 2: Pre Procesado de Datos

## Carga de los ficheros

Cargamos los ficheros de nuestro Google Drive personal

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/Colab_Notebooks/Bert_udemy/Aplicacion_dos_embedding_BERT/training.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [ ]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

## Pre Procesado

### Limpieza

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Removing the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Removing the URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Keeping only letters
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Removing additional whitespaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [ ]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [ ]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

Pasar los datos filtrados a un text

In [ ]:
for i in range(15):
    algo = f"{data_labels[i]}, {data_clean[i]}"
    print(algo)

In [ ]:
print(len(data_labels))
print(len(data_clean))

1600000
1600000


In [ ]:
archivo = open("/content/drive/MyDrive/Colab_Notebooks/Bert_udemy/Aplicacion_dos_embedding_BERT/data_clean.txt",'w')
for i in range(1600000):
    algo = f"{data_labels[i]}, {data_clean[i]}"
    archivo.write(algo)
    archivo.write("\n")
    
archivo.close()

### Tokenización

Necesitamos crear una capa BERT para tener acceso a los metadatos del tokenizador (como el tamaño del vocabulario).

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

Solo usamos la primera oración para las entradas BERT, por lo que agregamos el token CLS al principio y el token SEP al final de cada oración.

In [ ]:
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [ ]:
print(data_clean[1:3])

["is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!", ' I dived many times for the ball. Managed to save The rest go out of bounds']


In [ ]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

### Creación del dataset

Necesitamos crear las 3 entradas diferentes para cada oración.

In [ ]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # convierte los 1 en 0 y vice versa
    return seg_ids

Crearemos padded batches (por lo que rellenamos las frases para cada lote de forma independiente), de esta forma añadimos el mínimo número de tokens de padding posible. Para eso, ordenamos las frases por longitud, aplicamos padded_batches y luego las mezclamos.

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
               sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [ ]:
# Una lista es un tipo de iterador de modo que se puede usar como generador de un dataset
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [ ]:
NB_BATCHES = math.ceil((len(sorted_all) / BATCH_SIZE)*0.288)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [ ]:
print(NB_BATCHES )

In [ ]:
next(iter(train_dataset))

In [ ]:
my_sent = ["[CLS]"] + tokenizer.tokenize("Roses are red.") + ["[SEP]"]
print(my_sent)

['[CLS]', 'roses', 'are', 'red', '.', '[SEP]']


In [ ]:
bert_layer([tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32), 0)])

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-9.27935600e-01, -4.10335362e-01, -9.65754867e-01,
          9.07317877e-01,  8.12913775e-01, -1.74174309e-01,
          9.11234498e-01,  3.41952324e-01, -8.74521255e-01,
         -9.99989271e-01, -7.78410017e-01,  9.69385266e-01,
          9.86160576e-01,  6.36963069e-01,  9.48631287e-01,
         -7.51193106e-01, -4.58339393e-01, -7.08104432e-01,
          4.62098300e-01, -6.57927155e-01,  7.60414660e-01,
          9.99994814e-01, -3.96860808e-01,  3.44166070e-01,
          6.16488814e-01,  9.94400024e-01, -7.76633799e-01,
          9.38316524e-01,  9.59452271e-01,  7.32879460e-01,
         -6.93436682e-01,  2.93080449e-01, -9.93785560e-01,
         -1.64551765e-01, -9.67019558e-01, -9.95549619e-01,
          5.32935560e-01, -6.88061118e-01,  1.34715429e-02,
          2.98195258e-02, -9.18356538e-01,  4.20526177e-01,
          9.99988973e-01,  2.52676368e-01,  6.06235564e-01,
         -3.50750148e-01, -1.00000000e+00,  4.975

# Fase 3: Construcción del modelo

In [ ]:
class DCNNBERTEmbedding(tf.keras.Model):
    
    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=False)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def embed_with_bert(self, all_tokens):
        _, embs = self.bert_layer([all_tokens[:, 0, :],
                                   all_tokens[:, 1, :],
                                   all_tokens[:, 2, :]])
        return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)

        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Fase 4: Entrenamiento

In [ ]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 2

In [ ]:

Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Colab_Notebooks/Bert_udemy/Aplicacion_dos_embedding_BERT/checkponit"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

Último checkpoint restaurado!!


In [ ]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint guardado en {}.".format(checkpoint_path))
        print("Tiempo total para entrenar 1 epoch: {} segs\n".format(time.time() - start))

In [ ]:
Dcnn.build()

TypeError: ignored

In [ ]:
print(Dcnn.summary())

ValueError: ignored

## Result

In [ ]:
start = time.time()
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])

# Stage 5: Evaluation

In [ ]:
results = Dcnn.evaluate(test_dataset)
print(results)

In [ ]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)

    input_ids = get_ids(tokens)
    input_mask = get_mask(tokens)
    segment_ids = get_segments(tokens)

    inputs = tf.stack(
        [tf.cast(input_ids, dtype=tf.int32),
         tf.cast(input_mask, dtype=tf.int32),
         tf.cast(segment_ids, dtype=tf.int32)],
         axis=0)
    inputs = tf.expand_dims(inputs, 0) # simula un lote

    output = Dcnn(inputs, training=False)

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Salida del modelo: {}\nSentimiento predicho: Negativo.".format(
            output))
    elif sentiment == 1:
        print("Salida del modelo: {}\nSentimiento predicho: Positivo.".format(
            output))

In [ ]:
get_prediction("i feel strong and good overall")

Salida del modelo: [[0.97839725]]
Sentimiento predicho: Positivo.


In [ ]:
get_prediction("i feel like this was such a rude comment and im glad that t")

## Gardar el modelo

In [ ]:
#!pip install pyyaml h5py

In [ ]:
# se guarda el modelo
from tensorflow.keras.models import load_model

Dcnn.save('/content/drive/MyDrive/Colab_Notebooks/Bert_udemy/Aplicacion_dos_embedding_BERT/my_modelo/primer_modelo')

In [ ]:
import tensorflow as tf
#cargar el modelo
mio_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab_Notebooks/Bert_udemy/Aplicacion_dos_embedding_BERT/my_modelo/primer_modelo')

In [ ]:
Dcnn.summary()

Model: "dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  multiple                  109482241 
                                                                 
 conv1d (Conv1D)             multiple                  153700    
                                                                 
 conv1d_1 (Conv1D)           multiple                  230500    
                                                                 
 conv1d_2 (Conv1D)           multiple                  307300    
                                                                 
 global_max_pooling1d (Globa  multiple                 0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               multiple                  77056     
                                                              

In [ ]:
!pip install bert-for-tf2
#!pip install sentencepiece

In [ ]:
import bert

In [ ]:
import tensorflow_hub as hub

#from tensorflow.keras import layers

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # convierte los 1 en 0 y vice versa
    return seg_ids

In [ ]:
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [ ]:
def get_predictioonn(sentence):
    tokens = encode_sentence(sentence)

    input_ids = get_ids(tokens)
    input_mask = get_mask(tokens)
    segment_ids = get_segments(tokens)

    inputs = tf.stack(
        [tf.cast(input_ids, dtype=tf.int32),
         tf.cast(input_mask, dtype=tf.int32),
         tf.cast(segment_ids, dtype=tf.int32)],
         axis=0)
    inputs = tf.expand_dims(inputs, 0) # simula un lote

    output = mio_model(inputs, training=False)

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Salida del modelo: {}\nSentimiento predicho: Negativo.".format(
            output))
    elif sentiment == 1:
        print("Salida del modelo: {}\nSentimiento predicho: Positivo.".format(
            output))

In [ ]:
get_predictioonn("i feel strong and good overall")

Salida del modelo: [[0.97839725]]
Sentimiento predicho: Positivo.
